In [2]:
companies = [
    'MM',
    'NESTLEIND',
    'NTPC',
    'ONGC',
    'POWERGRID',
    'RELIANCE',
    'SBIN',
    'SHREECEM',
    'SUNPHARMA',
    'TATAMOTORS',
    'TATASTEEL',
    'TCS',
    'TECHM',
    'TITAN',
    'ULTRACEMCO',
    'UPL',
    'VEDL',
    'WIPRO',
    'ZEEL'
]

print(len(companies))

19


In [3]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D
import numpy as np
import pandas as pd
import keras
from keras.callbacks import EarlyStopping
from keras import backend as K

In [4]:
for company in companies:
    K.clear_session()
    df = pd.read_csv(f'stocks_new/{company}.csv')

    prices = df['Close'].values.tolist()

    def clean_stock_prices(stock_prices):
        cleaned_prices = []
        for price in stock_prices:

            if isinstance(price, str):
                price = price.replace(',', '').strip()  # Remove commas
            try:
                price = float(price)
                if not np.isnan(price):  # Check for NaN values
                    cleaned_prices.append(price)
            except ValueError:
                pass  # Skip invalid values

        return np.array(cleaned_prices).reshape(-1,1)

    prices = clean_stock_prices(prices)

    scaler = MinMaxScaler()
    scaled_prices = scaler.fit_transform(prices)

    def create_sequences(data, sequence_length):
        sequences = []
        labels = []
        for i in range(len(data) - (sequence_length+30)):
            sequences.append(data[i:i+sequence_length])
            labels.append(data[i+sequence_length+30])

        return np.array(sequences), np.array(labels)

    sequence_length = 60
    X, y = create_sequences(scaled_prices, sequence_length)

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Building the LSTM model
    model = Sequential([
        Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        MaxPooling1D(pool_size=2),
        LSTM(256, return_sequences=True),
        Dropout(0.2),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dropout(0.1),
        Dense(1)  # Output layer for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

    # Train the model
    print(f'Training {company} model...')
    history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=0)

    model.save(f'models\{company}_model.h5')
    print(f'{company} model saved')

    

Training MM model...
MM model saved
Training NESTLEIND model...
NESTLEIND model saved
Training NTPC model...
NTPC model saved
Training ONGC model...
ONGC model saved
Training POWERGRID model...
POWERGRID model saved
Training RELIANCE model...
RELIANCE model saved
Training SBIN model...
SBIN model saved
Training SHREECEM model...
SHREECEM model saved
Training SUNPHARMA model...


KeyboardInterrupt: 

In [64]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D
import numpy as np

import pandas as pd

import keras
print(keras.__version__)
print(keras.backend.backend())
# output should be "tensorflow"

df = pd.read_csv('stocks_new/BPCL.csv')

prices = df['Close'].values.tolist()

def clean_stock_prices(stock_prices):
    cleaned_prices = []
    for price in stock_prices:

        if isinstance(price, str):
            price = price.replace(',', '').strip()  # Remove commas
        try:
            price = float(price)
            if not np.isnan(price):  # Check for NaN values
                cleaned_prices.append(price)
        except ValueError:
            pass  # Skip invalid values

    return np.array(cleaned_prices).reshape(-1,1)

prices = clean_stock_prices(prices)

scaler = MinMaxScaler()
scaled_prices = scaler.fit_transform(prices)

def create_sequences(data, sequence_length):
  sequences = []
  labels = []
  for i in range(len(data) - (sequence_length+30)):
    sequences.append(data[i:i+sequence_length])
    labels.append(data[i+sequence_length+30])

  return np.array(sequences), np.array(labels)

sequence_length = 60
X, y = create_sequences(scaled_prices, sequence_length)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

2.10.0
tensorflow


In [65]:
nan_count = np.isnan(prices).sum()
print(f"Number of NaN values in 'prices': {nan_count}")
print(prices[10])

Number of NaN values in 'prices': 0
[9.79]


In [66]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Building the LSTM model
model = Sequential([
    Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    LSTM(256, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.1),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/500
162/162 [==============================] - 8s 26ms/step - loss: 0.0058 - val_loss: 0.0028
Epoch 2/500
162/162 [==============================] - 3s 20ms/step - loss: 0.0028 - val_loss: 0.0041
Epoch 3/500
162/162 [==============================] - 3s 20ms/step - loss: 0.0027 - val_loss: 0.0028
Epoch 4/500
162/162 [==============================] - 3s 19ms/step - loss: 0.0025 - val_loss: 0.0028
Epoch 5/500
162/162 [==============================] - 3s 19ms/step - loss: 0.0023 - val_loss: 0.0020
Epoch 6/500
162/162 [==============================] - 3s 19ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 7/500
162/162 [==============================] - 3s 19ms/step - loss: 0.0024 - val_loss: 0.0027
Epoch 8/500
162/162 [==============================] - 3s 19ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 9/500
162/162 [==============================] - 3s 18ms/step - loss: 0.0024 - val_loss: 0.0021
Epoch 10/500
162/162 [==============================] - 3s 19ms/step - loss: 0.002

In [67]:
model.save('models\BPCL_model.h5')

print(f"Model saved")

Model saved


In [68]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,mean_absolute_percentage_error

p_test = model.predict(X_test)
p_test = scaler.inverse_transform(p_test)
actual_prices = scaler.inverse_transform(y_test.reshape(-1,1))

r2 = r2_score(actual_prices, p_test)
mse = mean_squared_error(actual_prices, p_test)
mae = mean_absolute_error(actual_prices, p_test)
mape = mean_absolute_percentage_error(actual_prices, p_test)

print(f'R-squared (R²): {r2}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}')


45/45 [==============================] - 1s 10ms/step
R-squared (R²): 0.9974736343647608
Mean Squared Error (MSE): 18.97243859006197
Mean Absolute Error (MAE): 3.1950845576429203
Mean Absolute Percentage Error (MAPE): 0.08417434335709627
